In [2]:
# since you first have to create a virtual environment it get created with no jupyter notebooks preinstalled, so:
# pip install --user ipykernel - this install the jupyter kernel 
# python -m ipykernel install --user --name=myenv - this creates the environment as kernel
#I think there is no need to do have a virtual environment heret since it's just a notebook with analysis, 

C:\Users\ModestasMotiejunas\anaconda3\python.exe

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
anaconda-project 0.10.2 requires ruamel-yaml, which is not installed.



  Attempting uninstall: requests
    Found existing installation: requests 2.27.1
    Uninstalling requests-2.27.1:
      Successfully uninstalled requests-2.27.1
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.8.1
    Uninstalling aiohttp-3.8.1:
      Successfully uninstalled aiohttp-3.8.1


In [6]:

from chessdotcom import client, get_player_game_archives
import pprint
import requests
import pandas as pd

username = 'chiopra'

printer = pprint.PrettyPrinter()

def print_stats(username):
    data = client.get_player_stats(username, tts=0)
    printer.pprint(data.json)



{'stats': {'chess_blitz': {'best': {'date': 1437375600,
                                    'game': 'https://www.chess.com/game/live/64361468515',
                                    'rating': 1200},
                           'last': {'date': 1671976701,
                                    'rating': 918,
                                    'rd': 22},
                           'record': {'draw': 32, 'loss': 521, 'win': 553}},
           'chess_bullet': {'last': {'date': 1669305049,
                                     'rating': 871,
                                     'rd': 202},
                            'record': {'draw': 0, 'loss': 3, 'win': 0}},
           'chess_rapid': {'best': {'date': 1677690074,
                                    'game': 'https://www.chess.com/game/live/71428923535',
                                    'rating': 1287},
                           'last': {'date': 1677691265,
                                    'rating': 1278,
                              

### A/B Test: Playing more than 4 games per day leads to better results

In [18]:
def get_games(username):
    data = get_player_game_archives(username)
    for key,value in data.json.items():
        for url in value:
            games = requests.get(url).json()
            print(url)
            print()
            print()
            print()
#             print(games)
            printer.pprint(games['games'])
            break;
#             for game in games['games']:
#                 print(game['time_class'])
#                 print(type(game))
#                 df = pd.DataFrame.from_dict(game, orient = 'columns')
#                 print(df)
#                 print
#                 print(df.columns)
#                 string = df['pgn'].to_string(index=False)
#                 print(string)
#                 for i in string.split('"'):
#                     print(i)
#                 break;
#             break;
              
    # for i in data['Archives']:
    #     print(i)
    # printer.pprint(data.json)

get_games(username)

# print(requests.get('https://www.chess.com/game/live/1221711014').content())

https://api.chess.com/pub/player/chiopra/games/2015/07



[{'black': {'@id': 'https://api.chess.com/pub/player/pokemon225',
            'rating': 1256,
            'result': 'win',
            'username': 'pokemon225',
            'uuid': 'ea06eb9e-2f05-11e5-801e-000000000000'},
  'end_time': 1437418494,
  'fen': 'r1b1k1nr/ppp2ppp/4p3/2b5/8/8/PPP2PPP/R1BK1BNR w kq -',
  'initial_setup': 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1',
  'pgn': '[Event "Live Chess"]\n'
         '[Site "Chess.com"]\n'
         '[Date "2015.07.20"]\n'
         '[Round "-"]\n'
         '[White "chiopra"]\n'
         '[Black "pokemon225"]\n'
         '[Result "0-1"]\n'
         '[CurrentPosition "r1b1k1nr/ppp2ppp/4p3/2b5/8/8/PPP2PPP/R1BK1BNR w kq '
         '-"]\n'
         '[Timezone "UTC"]\n'
         '[ECO "C00"]\n'
         '[ECOUrl '
         '"https://www.chess.com/openings/French-Defense-Normal-Variation"]\n'
         '[UTCDate "2015.07.20"]\n'
         '[UTCTime "18:53:43"]\n'
         '[